In [119]:
import pandas as pd
import numpy as np

In [137]:
data = pd.read_csv("Agrofood_co2_emission.csv")

In [139]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6965 entries, 0 to 6964
Data columns (total 31 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Area                             6965 non-null   object 
 1   Year                             6965 non-null   int64  
 2   Savanna fires                    6934 non-null   float64
 3   Forest fires                     6872 non-null   float64
 4   Crop Residues                    5576 non-null   float64
 5   Rice Cultivation                 6965 non-null   float64
 6   Drained organic soils (CO2)      6965 non-null   float64
 7   Pesticides Manufacturing         6965 non-null   float64
 8   Food Transport                   6965 non-null   float64
 9   Forestland                       6472 non-null   float64
 10  Net Forest conversion            6472 non-null   float64
 11  Food Household Consumption       6492 non-null   float64
 12  Food Retail         

* Imputing for missing values (crop residues has the maximum NA values. It has a 0.8 corelation w the total emmission)
* PCA food_columns = ['food transport', 'food retail', 'food processing', 'food packaging', 'ippu', 'food household consumption']
* Average out: Manure applied to Soils, Manure left on Pasture, Manure Management
* Population
* Fires PCA
* drop avg temp
* Weighted average: On-farm energy use, On-farm Electricity Use

In [122]:
from sklearn.impute import SimpleImputer
imp = SimpleImputer(strategy='mean')
area = data['Area']
columns=data.columns
columns = columns.drop('Area')
data = imp.fit_transform(data.drop(['Area'], axis=1))
data = pd.DataFrame(data, columns=columns)
data['Area'] = area

### Test 1: without PCA

In [123]:
from sklearn.decomposition import PCA, SparsePCA
pca = PCA(n_components=1)
sparsePCA = SparsePCA(n_components=1)

In [124]:
food_columns = ['Food Transport', 'Food Retail', 'Food Processing', 'Food Packaging', 'IPPU', 'Food Household Consumption']

data['food_pca'] = pca.fit_transform(data[food_columns])
data = data.drop(['Food Transport', 'Food Retail', 'Food Processing', 'Food Packaging', 'IPPU', 'Food Household Consumption'], axis=1)
# data

In [125]:
manure = ['Manure applied to Soils', 'Manure left on Pasture', 'Manure Management']

data['Manure_pca'] = pca.fit_transform(data[manure])
data = data.drop(['Manure applied to Soils', 'Manure left on Pasture', 'Manure Management'], axis=1)
# data

In [126]:
data['Population'] = data[['Total Population - Male', 'Total Population - Female']].sum(axis=1)
data = data.drop(['Total Population - Male','Total Population - Female','Rural population','Urban population'], axis=1)
# data

In [127]:
fires = ['Savanna fires', 'Forest fires', 'Fires in organic soils', 'Fires in humid tropical forests']

data['Fires'] = pca.fit_transform(data[fires])
data = data.drop(['Savanna fires', 'Forest fires', 'Fires in organic soils', 'Fires in humid tropical forests'], axis=1)
# data

In [128]:
data = data.drop(['Average Temperature °C'], axis=1)
# data

In [129]:
data['Energy Consumption'] = (data['On-farm energy use'] + data['On-farm Electricity Use'])/2
data = data.drop(['On-farm energy use','On-farm Electricity Use'], axis=1)
# data

In [130]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
years = data['Year']
columns=data.columns
columns = columns.drop(['Area','Year'])
data = data.drop(['Area', 'Year'], axis=1)
data = scaler.fit_transform(data)
data = pd.DataFrame(data, columns=columns)
data['Area'] = area
data['Year'] = years

In [136]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
Crop Residues,6965.0,0.029821,0.098860,0.0,0.000757,0.005765,0.029821,1.0
Rice Cultivation,6965.0,0.025829,0.106805,0.0,0.001099,0.003243,0.009318,1.0
Drained organic soils (CO2),6965.0,0.014535,0.065808,0.0,0.000000,0.000000,0.002864,1.0
Pesticides Manufacturing,6965.0,0.020258,0.086831,0.0,0.000365,0.000790,0.007068,1.0
Forestland,6965.0,0.804866,0.081465,0.0,0.817122,0.823145,0.823278,1.0
Net Forest conversion,6965.0,0.010969,0.060751,0.0,0.000000,0.000078,0.006154,1.0
Agrifood Systems Waste Disposal,6965.0,0.028216,0.103881,0.0,0.000405,0.004224,0.014094,1.0
Fertilizers Manufacturing,6965.0,0.017771,0.068450,0.0,0.002109,0.006527,0.011853,1.0
total_emission,6965.0,0.130019,0.065102,0.0,0.113232,0.115207,0.121763,1.0
food_pca,6965.0,0.010909,0.057096,0.0,0.000075,0.000799,0.007699,1.0


In [138]:
data_IN = data[data['Area'] == "India"]
data_IN.drop('Area', axis=1, inplace=True)
data_IN = data_IN.set_axis(data_IN['Year'])
data_IN.drop('Year',axis=1).T

C:\Users\aryan\AppData\Local\Temp\ipykernel_456\2622582087.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_IN.drop('Area', axis=1, inplace=True)


Year,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
Savanna fires,3.337507e+02,3.337507e+02,3.337507e+02,3.337507e+02,3.337507e+02,3.337507e+02,2.037261e+02,1.402193e+02,2.286617e+02,2.542614e+02,...,3.795638e+02,6.459130e+02,2.932222e+02,3.319609e+02,2.435351e+02,3.775075e+02,4.279975e+02,4.733602e+02,3.058672e+02,1.945101e+02
Forest fires,4.560859e+03,4.560859e+03,4.560859e+03,4.560859e+03,4.560859e+03,4.560859e+03,1.788646e+03,1.538400e+03,1.561679e+03,3.236414e+03,...,4.163209e+03,9.862510e+03,5.697539e+03,4.786213e+03,3.586341e+03,4.711584e+03,7.451467e+03,6.052726e+03,3.516595e+03,1.762623e+03
Crop Residues,1.534452e+04,1.531279e+04,1.572680e+04,1.614243e+04,1.626249e+04,1.635811e+04,1.678007e+04,1.722014e+04,1.736153e+04,1.784379e+04,...,2.092966e+04,2.117115e+04,2.131248e+04,2.134715e+04,2.063471e+04,2.158447e+04,2.236681e+04,2.263915e+04,2.274067e+04,2.352288e+04
Rice Cultivation,1.261680e+05,1.260559e+05,1.234735e+05,1.257317e+05,1.265445e+05,1.265031e+05,1.282765e+05,1.284828e+05,1.324213e+05,1.334785e+05,...,1.300795e+05,1.263671e+05,1.304517e+05,1.303750e+05,1.282470e+05,1.276558e+05,1.293821e+05,1.305123e+05,1.293997e+05,1.330056e+05
Drained organic soils (CO2),5.007914e+03,5.007914e+03,5.007914e+03,5.007914e+03,5.007914e+03,5.023766e+03,5.022472e+03,5.026134e+03,5.025491e+03,5.010651e+03,...,4.960014e+03,4.958208e+03,4.957413e+03,4.953238e+03,4.952049e+03,4.945176e+03,4.945166e+03,4.946166e+03,4.906032e+03,4.902237e+03
Pesticides Manufacturing,2.401000e+03,2.349000e+03,2.363000e+03,2.204000e+03,2.006000e+03,2.051000e+03,2.027000e+03,1.822000e+03,1.797000e+03,1.795000e+03,...,1.753000e+03,1.771000e+03,1.485000e+03,1.864000e+03,1.798000e+03,1.789000e+03,1.933000e+03,1.843000e+03,1.872000e+03,1.872000e+03
Food Transport,9.715177e+03,1.025359e+04,1.012096e+04,1.026365e+04,1.053444e+04,1.080220e+04,1.270299e+04,1.395656e+04,1.384862e+04,1.439995e+04,...,8.257810e+03,8.475039e+03,2.470517e+04,2.567901e+04,2.816065e+04,2.936006e+04,1.134180e+04,1.523403e+04,1.427339e+04,1.213941e+04
Forestland,-2.273969e+05,-2.273969e+05,-2.273969e+05,-2.273969e+05,-2.273969e+05,-2.273969e+05,-2.273969e+05,-2.273969e+05,-2.273969e+05,-2.273969e+05,...,-7.111544e+04,-7.111544e+04,-7.111544e+04,-7.111544e+04,-7.111544e+04,-4.053720e+04,-4.053720e+04,-4.053720e+04,-4.053720e+04,-4.053720e+04
Net Forest conversion,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
Food Household Consumption,7.236799e+03,7.952514e+03,8.732251e+03,9.462371e+03,1.023454e+04,1.088609e+04,1.231418e+04,1.362616e+04,1.612503e+04,1.716853e+04,...,4.720452e+04,4.941696e+04,5.509266e+04,5.998270e+04,6.276369e+04,6.750432e+04,7.275201e+04,7.499256e+04,7.958059e+04,8.305574e+04
